### 1. Download the Data from Kaggle and Unzip


In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d 'maedemaftouni/large-covid19-ct-slice-dataset'

In [ ]:
import shutil
from google.colab import drive

drive.mount("/content/gdrive") # Connect to google drive as we will save model weights here
shutil.unpack_archive("/content/large-covid19-ct-slice-dataset.zip", "/tmp/")

### 1. OR Unzip the data if you have uploaded it to drive

In [4]:
import shutil
from google.colab import drive

drive.mount("/content/gdrive")
# Change the code below if the path to the dataset is different for you.
shutil.unpack_archive("/content/gdrive/MyDrive/archive.zip", "/tmp/")

Mounted at /content/gdrive


### 2. Splitting

In [5]:
import pandas as pd

meta_normal = pd.read_csv("/tmp/meta_data_normal.csv")
meta_covid = pd.read_csv("/tmp/meta_data_covid.csv", encoding='windows-1252')
meta_cap = pd.read_csv("/tmp/meta_data_cap.csv")

# Define the variables below using meta dataframes

normal_pt_nb = len(pd.unique(meta_normal["Patient ID"])) ##### INSERT YOUR CODE HERE ##### # Number of patients in normal group
covid_pt_nb = len(pd.unique(meta_covid["Patient ID"])) ##### INSERT YOUR CODE HERE ##### # Number of patients in covid group
cap_pt_nb = len(pd.unique(meta_cap["Patient ID"])) ##### INSERT YOUR CODE HERE ##### # Number of patients in CAP group

normal_img_nb = len(meta_normal) ##### INSERT YOUR CODE HERE ##### # Number of images in normal group
covid_img_nb = len(meta_covid) ##### INSERT YOUR CODE HERE ##### # Number of images in covid group
cap_img_nb = len(meta_cap) ##### INSERT YOUR CODE HERE ##### # Number of images in CAP group

print("Normal")
print("Number of patient: ", normal_pt_nb)
print("Number of image: ", normal_img_nb)

print("\nCovid")
print("Number of patient: ", covid_pt_nb)
print("Number of image: ", covid_img_nb)

print("\nCAP")
print("Number of patient: ", cap_pt_nb)
print("Number of image: ", cap_img_nb)


Normal
Number of patient:  604
Number of image:  6893

Covid
Number of patient:  464
Number of image:  7593

CAP
Number of patient:  54
Number of image:  2618


In [6]:

import numpy as np

# Set seed to get the same result (I specifically chose this seed after a couple of tries so that we'll have approximately same split ratios on slice level as well)
np.random.seed(58)
val_split_size = .2
test_split_size = .5

normal_val_file_list, normal_test_file_list = [], []
covid_val_file_list, covid_test_file_list = [], []
##### START OF YOUR CODE #####
normal_pt_list = np.array(pd.unique(meta_normal["Patient ID"]))
np.random.shuffle(normal_pt_list)
normal_val_pts = normal_pt_list[:int(normal_pt_nb * val_split_size)]
normal_test_pts = normal_pt_list[int(normal_pt_nb * val_split_size): int(normal_pt_nb * (val_split_size+test_split_size))]

normal_val_file_list = []
for pt in normal_val_pts:
  for i in meta_normal["File name"][meta_normal["Patient ID"] == pt].values:
    normal_val_file_list.append(i)

normal_test_file_list = []
for pt in normal_test_pts:
  for i in meta_normal["File name"][meta_normal["Patient ID"] == pt].values:
    normal_test_file_list.append(i)

covid_pt_list = np.array(pd.unique(meta_covid["Patient ID"]))
np.random.shuffle(covid_pt_list)
covid_val_pts = covid_pt_list[:int(covid_pt_nb * val_split_size)]
covid_test_pts = covid_pt_list[int(covid_pt_nb * val_split_size): int(covid_pt_nb * (val_split_size+test_split_size))]

covid_val_file_list = []
for pt in covid_val_pts:
  for i in meta_covid["File name"][meta_covid["Patient ID"] == pt].values:
    covid_val_file_list.append(i)

covid_test_file_list = []
for pt in covid_test_pts:
  for i in meta_covid["File name"][meta_covid["Patient ID"] == pt].values:
    covid_test_file_list.append(i)

##### END OF YOUR CODE #####

print("Slice-based val size: ")
print("Normal: ", round(len(normal_val_file_list)/normal_img_nb, 2))
print("Covid: ", round(len(covid_val_file_list)/covid_img_nb, 2))

print("\nSlice-based test size: ")
print("Normal: ", round(len(normal_test_file_list)/normal_img_nb, 2))
print("Covid: ", round(len(covid_test_file_list)/covid_img_nb, 2))

Slice-based val size: 
Normal:  0.21
Covid:  0.18

Slice-based test size: 
Normal:  0.51
Covid:  0.5


In [7]:
import os

##### START OF YOUR CODE #####
# Create train and test directories and move the files accordingly
[os.makedirs("/tmp/curated_data/data/"+x+y, exist_ok=True) for x in ["train/", "val/", "test/"] for y in ["normal/", "covid/"]]

# Normal group
for pt in os.listdir("/tmp/curated_data/curated_data/1NonCOVID/"):
  # validation
  if pt in normal_val_file_list:
    shutil.move("/tmp/curated_data/curated_data/1NonCOVID/"+pt, "/tmp/curated_data/data/val/normal/")
  # test
  elif pt in normal_test_file_list:
    shutil.move("/tmp/curated_data/curated_data/1NonCOVID/"+pt, "/tmp/curated_data/data/test/normal/")
  # train
  else:
    shutil.move("/tmp/curated_data/curated_data/1NonCOVID/"+pt, "/tmp/curated_data/data/train/normal/")

# Covid group
for pt in os.listdir("/tmp/curated_data/curated_data/2COVID/"):
  # validation
  if pt in covid_val_file_list:
    shutil.move("/tmp/curated_data/curated_data/2COVID/"+pt, "/tmp/curated_data/data/val/covid/")
  # test
  elif pt in covid_test_file_list:
    shutil.move("/tmp/curated_data/curated_data/2COVID/"+pt, "/tmp/curated_data/data/test/covid/")
  # train
  else:
    shutil.move("/tmp/curated_data/curated_data/2COVID/"+pt, "/tmp/curated_data/data/train/covid/")

##### END OF YOUR CODE #####

data_counts = {x+y: len(os.listdir("/tmp/curated_data/data/"+x+y)) for x in ["train/", "val/", "test/"] for y in ["normal/", "covid/"]}
for i in ["train", "val", "test"]:
  print("\nPercentage of {} set: {:.2f}" .format(i, (data_counts[i+"/normal/"]+data_counts[i+"/covid/"])/sum(data_counts.values())))
  print("Percentage of Covid + slices in {} set is: {:.2f}" .format(i, data_counts[i+"/covid/"]/(data_counts[i+"/normal/"]+data_counts[i+"/covid/"])))


Percentage of train set: 0.30
Percentage of Covid + slices in train set is: 0.55

Percentage of val set: 0.19
Percentage of Covid + slices in val set is: 0.49

Percentage of test set: 0.51
Percentage of Covid + slices in test set is: 0.52


### 3. Dataloader

In [8]:
!pip install -q torchio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.8/172.8 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 MB 16.0 MB/s eta 0:00:00


In [10]:
import os
import numpy as np
import torch
from PIL import Image
import torchio as tio
import torchvision.transforms as T


class CustomDataset(torch.utils.data.Dataset):
  def __init__(self, image_size, data_folder, partition):
    ##### START OF YOUR CODE #####
    self.image_size = image_size
    self.partition = partition
    self.data_folder = data_folder
    self.paths = self.img_paths()

    ##### END OF YOUR CODE #####
  def __len__(self):

    ##### START OF YOUR CODE #####
    return len(self.paths)
    ##### END OF YOUR CODE #####

  def __getitem__(self, idx):

    ##### START OF YOUR CODE #####
    img = self.read_and_resize_img(self.paths[idx])
    label = int(self.paths[idx].rsplit("/", 2)[1] == "covid")
    label = np.array(label)[np.newaxis]

    if self.partition == "train":
        img = self.augmentation(img)
    return (img, label)

    ##### END OF YOUR CODE #####

  def img_paths(self):
    ##### START OF YOUR CODE #####
    normal_paths = [os.path.join(self.data_folder, self.partition, "normal", i) for i in os.listdir(os.path.join(self.data_folder, self.partition, "normal"))]
    covid_paths = [os.path.join(self.data_folder, self.partition, "covid", i) for i in os.listdir(os.path.join(self.data_folder, self.partition, "covid"))]
    paths = normal_paths + covid_paths
    np.random.shuffle(paths)
    return paths

    ##### END OF YOUR CODE #####

  def read_and_resize_img(self, path):
    ##### START OF YOUR CODE #####
    img = Image.open(path).convert('L')
    img = np.array(img)[np.newaxis]/255.

    if (self.image_size, self.image_size) != img.shape[1:]:
      resizing = T.Resize((self.image_size, self.image_size), antialias=False)
      resized_img = resizing(torch.Tensor(img))
      return resized_img

    return torch.Tensor(img)

    ##### END OF YOUR CODE #####

  def augmentation(self, data):
    ##### START OF YOUR CODE #####

    transform = tio.transforms.OneOf({
        tio.transforms.OneOf({
            tio.transforms.RandomNoise(): .25,
            tio.transforms.RandomBiasField(): .25,
            tio.transforms.RandomGhosting(): .25,
            tio.transforms.RandomSpike(): .25,
            tio.transforms.RandomAffine(degrees=10, scales=0., translation=0.): .25
        }): .8
    })
    aug_data = torch.squeeze(transform(torch.unsqueeze(data, -1)), -1)
    return aug_data


    ##### END OF YOUR CODE #####


### 4. ResNet-18

In [11]:
import torch
from torch import nn
from torch.nn import functional as F


class ConvBlock(nn.Module):
    def __init__(self, ch_in, ch_out, s, act):
      super(ConvBlock,self).__init__()
      # Initialize layers

      ##### START OF YOUR CODE #####
      super(ConvBlock,self).__init__()
      # Initialize layers
      if act == "relu":
          self.act_layer = nn.ReLU()
      elif act == "leaky_relu":
          self.act_layer = nn.LeakyReLU()
      elif act == "gelu":
          self.act_layer = nn.GELU()

      self.conv1 = nn.Conv2d(ch_in, ch_in, kernel_size=3, stride=s, padding=1)
      self.bn1 = nn.BatchNorm2d(ch_in)
      self.conv2 = nn.Conv2d(ch_in, ch_out, kernel_size=3, stride=1, padding=1)
      self.bn2 = nn.BatchNorm2d(ch_out)

      self.conv_sc = nn.Conv2d(ch_in, ch_out, kernel_size=1, stride=s)
      self.bn_sc = nn.BatchNorm2d(ch_out)


      ##### END OF YOUR CODE #####

    def forward(self, X):
      ##### START OF YOUR CODE #####
      X_shortcut = X
      X = self.bn1(self.conv1(X))
      X = self.act_layer(X)
      X = self.bn2(self.conv2(X))

      X_shortcut = self.bn_sc(self.conv_sc(X_shortcut))
      X = self.act_layer(X + X_shortcut)


      ##### END OF YOUR CODE #####
      return X


class ResNet18(nn.Module):
    def __init__(self, act, drop_rate, image_size):
      super(ResNet18, self).__init__()
      # Initialize layers
      ##### START OF YOUR CODE #####

      if act == "relu":
          self.act_layer = nn.ReLU()
      elif act == "leaky_relu":
          self.act_layer = nn.LeakyReLU()
      elif act == "gelu":
          self.act_layer = nn.GELU()

      self.image_size = image_size

      self.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3) # 64 x N/2 x N/2
      self.bn1 = nn.BatchNorm2d(64)

      self.mp = nn.MaxPool2d((3, 3), stride=2, padding=1) # 64 x N/4 x N/4
      self.conv2a = ConvBlock(64, 64, 1, act)
      self.conv2b = ConvBlock(64, 64, 1, act) # 64 x N/4 x N/4

      self.conv3a = ConvBlock(64, 128, 2, act)
      self.conv3b = ConvBlock(128, 128, 1, act) # 128 x N/8 x N/8

      self.conv4a = ConvBlock(128, 256, 2, act)
      self.conv4b = ConvBlock(256, 256, 1, act) # 256 x N/16 x N/16

      self.conv5a = ConvBlock(256, 512, 2, act)
      self.conv5b = ConvBlock(512, 512, 1, act) # 512 x N/32 x N/32

      kernel_size = self.image_size /(2**5)
      self.avgpool = nn.AvgPool2d((int(kernel_size), int(kernel_size)))
      self.flat = nn.Flatten()
      self.dropout = nn.Dropout(drop_rate)
      self.fc = nn.Linear(512, 1)

      ##### END OF YOUR CODE #####

    def forward(self, X):
      ##### START OF YOUR CODE #####

      batch_size = X.size(0)
      image_size = X.size(2)

      X = self.conv1(X)
      X = self.bn1(X)
      X = self.act_layer(X)

      X = self.mp(X)
      X = self.conv2a(X)
      X = self.conv2b(X)

      X = self.conv3a(X)
      X = self.conv3b(X)

      X = self.conv4a(X)
      X = self.conv4b(X)

      X = self.conv5a(X)
      X = self.conv5b(X)

      X = self.avgpool(X)
      X = self.flat(X)
      X = self.fc(self.dropout(X))
      X = torch.sigmoid(X)

      ##### END OF YOUR CODE #####
      return X

In [12]:
# Print the model
model = ResNet18("relu", .5, 224)
print(model)

ResNet18(
  (act_layer): ReLU()
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (mp): MaxPool2d(kernel_size=(3, 3), stride=2, padding=1, dilation=1, ceil_mode=False)
  (conv2a): ConvBlock(
    (act_layer): ReLU()
    (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv_sc): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
    (bn_sc): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv2b): ConvBlock(
    (act_layer): ReLU()
    (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn1): BatchNorm2d(64, eps=1e-05, mo

# Assignment 3
<p>In this assignment you will

* write helper functions
* train the model
* hyperparameter search using W&B

Read the comments carefully and insert your code where you see: <br><br><b>##### START OF YOUR CODE #####</b><br><br><b>##### END OF YOUR CODE #####</b><br><br>or for the inline codes you will see<br><br><b>##### INSERT YOUR CODE HERE #####</b>

#### I. AverageMeter
First we will write a helper function. AverageMeter is to calculate the mean of the running loss and accuracy.

*   It will have 2 functions which are reset and update.
*   reset will be called on initialization and set the attributes to 0.
*   update takes 2 arguments for the value and the size. It will add the value to the sum and the size to the count. Attribute "avg" (use this name) will also be updated as sum/count.

In [25]:
## YİĞİT SARIOĞLU
# STUDENT NO: 2022400354

class AverageMeter:
    """Computes and stores the average and current value"""

     ##### START OF YOUR CODE #####

    #constructor method: Initializes the object and calls the reset method.
    def __init__(self):
        self.reset()

    # resets the sum and count attributes to 0.
    def reset(self):
        self.sum = 0.0
        self.count = 0

    # updates the sum and count based on the given value and size
    def update(self, value, size):
        self.sum += value * size
        self.count += size
        self.avg = self.sum / self.count


      ##### END OF YOUR CODE #####

In [14]:
avg_meter = AverageMeter()
avg_meter.update(100, 5)
avg_meter.update(50, 5)
print(avg_meter.avg, avg_meter.count)

75.0 10


#### II. Train Loop
Now we will write the training and validation loops. Detailed instructions are given within the code.

In [29]:
device = torch.device('cuda')


def accuracy(outputs, labels):
    _, predicted = torch.max(outputs, 1)
    correct = (predicted == labels).sum().item()
    total = labels.size(0)
    acc = correct / total
    return acc


def training(train_loader, model, criterion, optimizer):
  # Let's start by initializing our AverageMeters.
  avg_meters = {'loss': AverageMeter(),
                'acc': AverageMeter()}

  # We will go through the train_loader.
  # Zero the gradients.
  # Make prediction.
  # Calculate the loss and the accuracy using prediction and labels.
  # Update the average meters.
  # Compute gradients and adjust learning weights.

  ##### START OF YOUR CODE #####

  # We will go through the train_loader.
  for inputs, labels in train_loader:
      # move inputs and labels to the device (GPU).
      inputs, labels = inputs.to(device), labels.to(device)


      # zero the gradients.
      optimizer.zero_grad()

      # makes predictions
      outputs = model(inputs)



      # convert labels to float to match the criterion.
      labels = labels.float()

      loss = criterion(outputs, labels)

      acc = accuracy(outputs, labels)


      # Update the average meters.
      avg_meters['loss'].update(loss.item(), inputs.size(0))
      avg_meters['acc'].update(acc, inputs.size(0))

      # compute gradients and adjust learning weights.
      loss.backward()
      optimizer.step()


  ##### END OF YOUR CODE #####

  return dict([('loss', avg_meters['loss'].avg),
                ('acc', avg_meters['acc'].avg)])




def validation(val_loader, model, criterion):
  avg_meters = {'loss': AverageMeter(),
                'acc': AverageMeter()}

  # Validation is almost the same but don't forget to turn the eval mode of the model and with torch no_grad.
  # You don't need to compute gradients or adjust learning weights for evaluation.

  ##### START OF YOUR CODE #####
  model.eval()  # set(model) to evaluation mode
  with torch.no_grad():  #
    for inputs, labels in val_loader:
      # move inputs and labels to the device (GPU).
      inputs, labels = inputs.to(device), labels.to(device)

      # prediction
      outputs = model(inputs)

       # convert labels to float
      labels = labels.float()



      # calculate the loss and the accuracy using prediction and labels.
      loss = criterion(outputs, labels)
      acc = accuracy(outputs, labels)


      # update the average meters.
      avg_meters['loss'].update(loss.item(), inputs.size(0))

      avg_meters['acc'].update(acc, inputs.size(0))




  ##### END OF YOUR CODE #####

  return dict([('loss', avg_meters['loss'].avg),
              ('acc', avg_meters['acc'].avg)])

We will use Weights & Biases for hyperparameter search. This will only be an introduction and we highly recommend you to read the <a href="https://docs.wandb.ai/?_gl=1*1xon9b*_ga*NDg5OTYzNTM3LjE2NzUwNjYzNjk.*_ga_JH1SJHJQXJ*MTY3Njc0MDEyNi4xMi4xLjE2NzY3NDAxMjguNTguMC4w">documentation</a> for more information.

In [16]:
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.2/249.2 kB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.2 MB/s eta 0:00:00


In [28]:
from torch import optim
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
import os

import wandb


def main():
  print("here i am 0")
  # Set the initial configuration
  initial_config = {
      "data_dir": "/tmp/curated_data/data/",
      "image_size": 128,
      "train_batch_size": 64,
      "val_batch_size": 32,
      "test_batch_size": 1,
      "activation": "relu",
      "drop_rate": .2,
      "optimizer": "Adam",
      "learning_rate": 1e-3,
      "l2_reg": 1e-4, # Weight decay
      "nb_epoch": 50,
      "early_stopping": 15, # trigger value for early stopping

  }

  # Using this configuration dictionary:
  # initialize wandb
  # Create a run directory in your drive ("/content/drive/MyDrive/CMPE_runs/" + the current run name that you'll get from wandb)
  # Create the model
  # Create dataloader dictionary with "train", "val", "test" keys
  # Define binary cross entropy loss
  # Define optimizer with weight decay
  # Set lr scheduler to ReduceLROnPlateau:
    # It will decrease the lr by .1 if the val_loss did not decrease > .01. The minimum lr value can be 1e-9.
  # Print train and val results and log them to wandb at the end of each epoch
  # Save best model weights to your run directory when the val accuracy is at least .01 better than the best val accuracy.
  # Set early stopping with the trigger in config["early_stopping"], monitoring val accuracy. config["early_stopping"] = -1 means no early stopping.
  # Print when a new model is saved or early stopping trigger is reached.
  # After the final epoch (or early stopping), load the best model weights and log the test results to wandb

  ##### START OF YOUR CODE #####

  # Initialize wandb
  wandb.login()
  wandb.init(config=initial_config, project="cmpe49t_project")



  # Create a run directory in your drive ("/content/drive/MyDrive/CMPE_runs/" + the current run name that you'll get from wandb)
  # run_dir = "/content/drive/MyDrive/CMPE49t/" + wandb.run.name


 # print("wandb run name :  " , wandb.run.name)
 # run_directory = wandb.run.dir
 # print("run directory is " ,run_directory)

# I have an error here, I could not solve it.. ı could not set the running directory here
  #running_directory = os.path.join("/content/drive/MyDrive/CMPE49t/", wandb.run.name)
  #wandb.run.dir = running_directory



  # Create the model
  model = ResNet18(wandb.config.activation, wandb.config.drop_rate, wandb.config.image_size)
  model.to(device)


  # Create dataloader dictionary with "train", "val", "test" keys
  data_dict = {
      "train": CustomDataset(wandb.config.image_size, wandb.config.data_dir, "train"),
      "val": CustomDataset(wandb.config.image_size, wandb.config.data_dir, "val"),
      "test": CustomDataset(wandb.config.image_size, wandb.config.data_dir, "test")
  }


  dataloader_dict = {x: DataLoader(data_dict[x], batch_size=wandb.config[f"{x}_batch_size"], shuffle=True)
  for x in ["train", "val", "test"]}

  # Define binary cross entropy loss
  criterion = nn.BCELoss()

  # Define optimizer with weight decay
  optimizer = optim.Adam(model.parameters(), lr=wandb.config.learning_rate, weight_decay=wandb.config.l2_reg)


  # Set lr scheduler to ReduceLROnPlateau
  scheduler = ReduceLROnPlateau(optimizer, 'min', patience=5, factor=0.1, min_lr=1e-9)

  # Training loop
  best_val_acc = 0.0
  early_stopping_counter = 0


  for epoch in range(wandb.config.nb_epoch):
      # Training
      model.train()
      train_results = training(dataloader_dict["train"], model, criterion, optimizer)

    # Check if 'input' and 'target' keys exist in train_results
      if 'input' in train_results and 'target' in train_results:
    # Convert input and target tensors to float
        input_data = train_results["input"].float()
        target_data = train_results["target"].float()

    # further processing with input_data and target_data
      else:
        print("Missing 'input' or 'target' key in train_results.")






      # Validation
      model.eval()
      val_results = validation(dataloader_dict["val"], model, criterion)

      # Log results to wandb
      wandb.log({"train_loss": train_results["loss"],
                   "train_acc": train_results["acc"],
                   "val_loss": val_results["loss"],
                   "val_acc": val_results["acc"],
                   "lr": optimizer.param_groups[0]['lr'],
                   "epoch": epoch})

      # Check for early stopping
      if wandb.config.early_stopping > 0 and val_results["acc"] > best_val_acc + 0.01:
          best_val_acc = val_results["acc"]
          early_stopping_counter = 0
          # Save best model weights
          torch.save(model.state_dict(), os.path.join(wandb.run.dir, "best_model.pth"))
          print(f"New best model saved at epoch {epoch} with val accuracy {best_val_acc:.4f}.")
      else:
          early_stopping_counter += 1


      if wandb.config.early_stopping > 0 and early_stopping_counter >= wandb.config.early_stopping:
          print(f"Early stopping triggered at epoch {epoch}.")
          break

      # Adjust learning rate using scheduler
      scheduler.step(val_results["loss"])

  # After the final epoch (or early stopping), load the best model weights
  model.load_state_dict(torch.load(os.path.join(wandb.run.dir, "best_model.pth")))



  # Log test results to wandb
  model.eval()
  test_results = validation(dataloader_dict["test"], model, criterion)
  wandb.log({"test_loss": test_results["loss"], "test_acc": test_results["acc"]})

  print("Training completed!")

  # [optional] finish the wandb run, necessary in notebooks
  wandb.finish()

  ##### END OF YOUR CODE #####

In [31]:
# Define the parameters that we will fine tune with, which are:
  # Activation function
  # Optimizer
  # Drop rate: should be chosen randomly from a uniform distribution between [0., 0.9]
  # Weight decay: should be chosen randomly from a uniform distribution between [0., 0.1]
  # Learning rate: should be chosen randomly from a uniform distribution between [0.0001, 0.1]

# definition of the parameters


# Define the parameters that we will fine-tune
parameter_dict = {
    "activation": {"values": ["relu", "leaky_relu", "gelu"]},
    "optimizer": {"values": ["Adam", "SGD"]},
    "drop_rate": {"min": 0.0, "max": 0.9},
    "l2_reg": {"min": 0.0, "max": 0.1},
    "learning_rate": {"min": 0.0001, "max": 0.1},
}



# Define the sweep configuration
sweep_config = {
    "method": "random",
    "metric": {"goal": "maximize", "name": "val_acc"},
    "parameters": parameter_dict,
    "name": "my_sweep",  # Customize the sweep name
    "project": "cmpe49t_project",  # Replace with your project name
}

sweep_config['parameters'] = parameter_dict

# Start the sweep
sweep_id = wandb.sweep(sweep=sweep_config, project="cmpe49t_project", entity="bounteam") ##### INSERT YOUR CODE HERE #####


Create sweep with ID: 36lb4mvm
Sweep URL: https://wandb.ai/bounteam/cmpe49t_project/sweeps/36lb4mvm


In [ ]:
wandb.agent(sweep_id, function=main)